In [1]:
import sys
sys.path.insert(9, '/Users/jarad/fake_folder')

from jb_libraries import *
%matplotlib inline

### [Pink anti-static bags](https://volcano.adafruit.com/volcano/ada_plm.php?zenAdminID=c2d3ee839d38936bafca30cd28172680#/Sku/10926/SkuWhereUsed)

* sku_id = 10926

In [2]:
pd.read_sql(
'''
SELECT
sku_id,
sku,
sku_name
FROM skus
WHERE sku LIKE 'C10926%'
''', db)

,sku_id,sku,sku_name
0,10926,C10926-001,"pink anti-stat bags 3"" x 4"" x 3mil +1'' header w/sombrero hang hole for SX Pro Line Auto Bagger..."


### From Dean

In [3]:
def get_contains(sku_id):
    
    skus = []
    
    result = pd.read_sql(
    '''
    SELECT
    pts.sku_id AS 'sku id',
    k.sku,
    k.bom_type AS bom
    FROM products_to_stuff pts
    JOIN skus k ON pts.sku_id = k.sku_id
    AND k.sku_status = 'working'
    WHERE pts.contains_sku_id = ''' + str(sku_id) + '''
    ''', db)
    
    skus = result['sku id'].values.tolist()
    
    sku_copy = skus
    
    for sku in sku_copy:
        contains = get_contains(sku)
        for k in contains:
            if not k in skus:
                skus.append(k)
                
    return skus
            

pink_skus = get_contains(10926)
        
if len(pink_skus) == len(set(pink_skus)):
    print('no dupes')
else:
    print('you have dupes')
    
print('count of skus: ', len(pink_skus))    

no dupes
count of skus:  225


In [4]:
skus_to_parts_main = pd.read_sql(
'''
SELECT
part_id AS 'part id'
FROM skus
WHERE sku_id IN '''+ str(tuple(pink_skus)) +'''
''', db)

combo_parts = pd.read_sql(
'''
SELECT
part_id AS 'part id'
FROM products_to_stuff
WHERE contains_part_id IN '''+ str(tuple(skus_to_parts_main['part id'])) +'''
''', db)

all_pink_parts = skus_to_parts_main['part id'].tolist() + combo_parts['part id'].tolist()

if len(all_pink_parts) == len(set(all_pink_parts)):
    print('no dupes')
else:
    print('you have dupes')
    
print('count of all parts: ', len(all_pink_parts))    
print('count of unique parts: ', len(set(all_pink_parts)))

you have dupes
count of all parts:  311
count of unique parts:  303


In [5]:
sales_main = pd.read_sql(
'''
SELECT

op.part_id AS 'part id',
pd.products_name AS 'part name',
IFNULL(k.sku_id, 0) AS 'sku id',
IFNULL(k.sku, 0) AS sku,
IFNULL(k.bom_type, 'combo') AS bom,
SUM(op.products_quantity) AS 'total qty'

FROM orders_products op

JOIN orders o ON op.orders_id = o.orders_id
AND DATE(o.date_purchased) BETWEEN DATE_SUB(CURDATE(), INTERVAL 91 DAY) AND CURDATE()
AND o.orders_status != 9 # fraud pending
AND o.orders_status != 10 # fraud confirmed
# leave in replacement orders

LEFT JOIN
(SELECT
part_id,
sku_id,
sku,
bom_type
FROM skus
WHERE sku_status = 'working'
GROUP BY part_id) k ON op.part_id = k.part_id # sometimes a single part has mulitple 'working' skus so we need to group by part id

JOIN products_description pd ON op.part_id = pd.part_id

WHERE op.part_id IN '''+ str(tuple(all_pink_parts)) +'''

GROUP BY op.part_id
''', db)

if sales_main[sales_main['part id'].duplicated()].empty:
    print('no dupes')
else:
    print('you have dupes')
    
print('count of unique skus: %i' % len(sales_main['part id'].unique()))    
print('total qty: {:,.0f}'.format(sales_main['total qty'].sum()))

no dupes
count of unique skus: 283
total qty: 184,949


In [6]:
print(sales_main['bom'].value_counts())
print('\n')
print('current auto bagger usage per week in hours:', sales_main['total qty'].sum()/90*7*10/3600)

pnp       202
combo      74
kitted      7
Name: bom, dtype: int64


current auto bagger usage per week in hours: 39.958117284


### Nice table

In [7]:
nice = sales_main.groupby('bom')[['total qty']].sum()
nice.loc['total'] = nice.sum()

nice['/ 90'] = nice['total qty']/90.0
nice['* 7'] = nice['/ 90'] * 7
nice['* 10'] = nice['* 7'] * 10
nice['/ 3600'] = nice['* 10']/3600

nice.format_(['n0','n0','n0','n0','n2'])

,Total Qty,/ 90,* 7,* 10,/ 3600
bom,,,,,
combo,"7,847",87,610,"6,103",1.70
kitted,402,4,31,313,0.09
pnp,"176,700","1,963","13,743","137,433",38.18
total,"184,949","2,055","14,385","143,849",39.96


### Check totals

In [8]:
all_boms = pd.read_sql(
'''
SELECT
bom_type AS bom,
COUNT(*) AS 'all part count'
FROM skus
WHERE sku_status = 'working'
GROUP BY bom_type
''', db)

all_boms.set_index('bom', inplace = True)

In [9]:
all_combos = pd.read_sql(
'''
SELECT
'combo' AS bom,
COUNT(*) AS 'all part count'
FROM parts
WHERE products_combo = 1
AND products_discontinued = 0
''', db)

all_combos.set_index('bom', inplace = True)

In [10]:
check_main = pd.concat([all_boms,
                       all_combos])

sales_check = sales_main.groupby('bom')[['part id']].count().rename(columns = {'part id':'pink part count'})

all_check = pd.merge(check_main,
                    sales_check,
                    how = 'left',
                    left_index = True,
                    right_index = True).fillna(0)
all_check.loc['total'] = all_check.sum()
all_check['pink parts as % of total'] = all_check['pink part count']/all_check['all part count']

In [11]:
all_check.format_(['n0','n0','p2'])

,All Part Count,Pink Part Count,Pink Parts As % Of Total
bom,,,
pnp,396,202,51.01%
kitted,359,7,1.95%
unspecified,"2,026",0,0.00%
Resale - No Labor,"1,499",0,0.00%
Resale - Rapid Prep,386,0,0.00%
combo,172,74,43.02%
total,"4,838",283,5.85%


### Write to excel

In [14]:
writer = pd.ExcelWriter('Qty Sold Data for Auto Bagger as of Feb 09, 2018.xlsx')
sales_main.to_excel(writer, 'last 90 days', index = False)
nice.to_excel(writer, 'last 90 days', startrow = 3, startcol = len(sales_main.columns) + 1)
all_check.to_excel(writer, 'last 90 days', startrow = 3, startcol = len(sales_main.columns) + len(nice.columns) + 3)
#writer.save()